In [ ]:
import os

import numpy as np
import pandas as pd
import json
import cv2
from sklearn.model_selection import train_test_split
import cv2

# tf imports
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras import backend as K

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    print(e)

loading images with labels

In [ ]:
f = open(".\data\measurements.json")
data_json = json.loads(f.read())["Measurements"]

vs = data_json.values()

df = pd.json_normalize(vs)

df.head()

In [ ]:
df.sort_values("Height")

In [ ]:
# testing images shaping
img = cv2.imread('.\data\RGBImages\RGB_270.png')
w, h, c = img.shape

img = img[ (w // 2 - 400 ) : (w // 2 + 400) , (h // 2 - 400 + 100)  : (h // 2 + 400 + 100) ]
plt.imshow(img)

In [ ]:

path = './data/DepthImages/'
all_images = []
y = []
for image_path in os.listdir(path):
    y.append(df.loc[df['Depth_Information'] == image_path].Height.item())
    img = cv2.imread(path + image_path, -1)
    img = img[ (w // 2 - 400 ) : (w // 2 + 400) , (h // 2 - 400 + 100)  : (h // 2 + 400 + 100) ]
    img = cv2.resize(img, (250, 250))
    all_images.append(img)

X = np.array(all_images)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)

In [ ]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(800, 800, 1)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))

model.add(Dense(1, activation='linear'))

model.compile(loss="mean_squared_error", optimizer="adam", metrics=[r2_keras])
model.summary()

In [ ]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)
history = model.fit(X_train, y_train,
          batch_size=32,
          epochs=30,
          verbose=2,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
plt.plot(history.history['loss'][4 : ])
plt.plot(history.history['val_loss'][4: ])
plt.title('model MSE')
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.predict([X_train[0]])

ResNet Implementation for regression: depth images height

https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba

!!LET OP NIET RUNNEN ZONDER GPU!!

In [40]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, \
    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

In [41]:
def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

In [42]:
def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides=(1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

In [43]:
def create_res_net(input_shape, regression):
    inputs = Input(shape=input_shape)
    num_filters = 64

    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)

    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j == 0 and i != 0), filters=num_filters)
        num_filters *= 2

    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(19, activation='softmax')(t)

    if regression:
        outputs = Dense(1, activation='linear')(t)

    model = Model(inputs, outputs)

    if regression:
        model.compile(
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_absolute_percentage_error']
        )
    else:
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

    return model

In [44]:
rn = create_res_net((250,250,1),True)

In [45]:
rn.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 250, 250, 1  0           []                               
                                )]                                                                
                                                                                                  
 batch_normalization_90 (BatchN  (None, 250, 250, 1)  4          ['input_4[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 conv2d_96 (Conv2D)             (None, 250, 250, 64  640         ['batch_normalization_90[0][0]'] 
                                )                                                           

In [49]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import datetime
import os

timestr = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
name = 'res_net_depth'+timestr

checkpoint_path = "checkpoints/"+name+"/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
os.system('mkdir {}'.format(checkpoint_dir))

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=0,
    save_best_only=True,
    monitor='val_loss',
    mode='min'
)
tensorboard_callback = TensorBoard(
    log_dir=f'tensorboard_logs/{name}',
    histogram_freq=1
)

X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

rn.fit(
    x=X_train,
    y=y_train,
    epochs=100,
    verbose=1,
    validation_data=(X_test, y_test),
    batch_size=16,
    callbacks=[cp_callback, tensorboard_callback]
)

(310, 250, 250)
Epoch 1/100
20/20 [==============================] - ETA: 0s - loss: 6.3316 - mean_absolute_percentage_error: 18.8602
Epoch 1: saving model to checkpoints/res_net_depth20221206-114207\cp-0001.ckpt


INFO:tensorflow:Assets written to: checkpoints/res_net_depth20221206-114207\cp-0001.ckpt\assets


INFO:tensorflow:Assets written to: checkpoints/res_net_depth20221206-114207\cp-0001.ckpt\assets


20/20 [==============================] - 34s 2s/step - loss: 6.3316 - mean_absolute_percentage_error: 18.8602 - val_loss: 30.5157 - val_mean_absolute_percentage_error: 27.8470
Epoch 2/100
20/20 [==============================] - ETA: 0s - loss: 4.7581 - mean_absolute_percentage_error: 14.9169
Epoch 2: saving model to checkpoints/res_net_depth20221206-114207\cp-0002.ckpt


INFO:tensorflow:Assets written to: checkpoints/res_net_depth20221206-114207\cp-0002.ckpt\assets


...

20/20 [==============================] - 34s 2s/step - loss: 7.2853 - mean_absolute_percentage_error: 22.4452 - val_loss: 6.9462 - val_mean_absolute_percentage_error: 19.0375
Epoch 13/100
20/20 [==============================] - ETA: 0s - loss: 4.1140 - mean_absolute_percentage_error: 15.3496
Epoch 13: saving model to checkpoints/res_net_depth20221206-114207\cp-0013.ckpt


INFO:tensorflow:Assets written to: checkpoints/res_net_depth20221206-114207\cp-0013.ckpt\assets


INFO:tensorflow:Assets written to: checkpoints/res_net_depth20221206-114207\cp-0013.ckpt\assets


20/20 [==============================] - 34s 2s/step - loss: 4.1140 - mean_absolute_percentage_error: 15.3496 - val_loss: 16.6070 - val_mean_absolute_percentage_error: 19.2101
Epoch 14/100
20/20 [==============================] - ETA: 0s - loss: 4.0297 - mean_absolute_percentage_error: 14.4007
Epoch 14: saving model to checkpoints/res_net_depth20221206-114207\cp-0014.ckpt


FailedPreconditionError: {{function_node __wrapped__SaveV2_dtypes_448_device_/job:localhost/replica:0/task:0/device:CPU:0}} Failed to rename: checkpoints/res_net_depth20221206-114207\cp-0014.ckpt\variables\variables_temp/part-00000-of-00001.data-00000-of-00001.tempstate15686874212932515873 to: checkpoints/res_net_depth20221206-114207\cp-0014.ckpt\variables\variables_temp/part-00000-of-00001.data-00000-of-00001 : The process cannot access the file because it is being used by another process.
; Broken pipe [Op:SaveV2]